### 基础

In [1]:
# 列表：由其他对象的引用组成的连续数组
a = 1
b = 'hello'

c = [a, b]
c

[1, 'hello']

In [2]:
# a变成了对2的引用
a = 2

# 但列表的引用对象仍是：1， hello
c

[1, 'hello']

In [3]:
# 列表的引用对象变为：1，world
c[1] = 'world'

# 但b的引用对象仍是world
b

'hello'

In [5]:
# 列表推导式：一般而言，列表推导式比for循环跟快速。
[i for i in range(10) if i % 2 == 0]

[0, 2, 4, 6, 8]

In [6]:
# zip函数：对zip函数返回的结果再次调用zip函数，可以使其恢复原状
for item in zip(*zip([1, 2, 3], [4, 5, 6])):
    print(item)

(1, 2, 3)
(4, 5, 6)


In [8]:
# 序列解包 -- *号的用法
first, *inner, last = 0, 1, 2, 3

print('first:', first)
print('inner:', inner)
print('last:', last)

first: 0
inner: [1, 2]
last: 3


In [9]:
# 嵌套解包
(a, b), (c, d) = (1, 2), (3, 4)
a, b, c, d

(1, 2, 3, 4)

In [10]:
# dict的视图对象（view objects）: keys()、values()和items()均返回视图对象
# 字典发生变化时，视图也会发生相应的变化
# 字典在CPython中使用hash table实现，所以只有hashable objects才能作为字典的键。
# python中所有不可变类型都是可哈希的
word = {'a': 1, 'b': 2}
items = word.items()
word['c'] = 3
items

dict_items([('a', 1), ('b', 2), ('c', 3)])

### 迭代器

In [11]:
# 迭代器：实现了迭代器协议的容器对象，包含一下2个方法
# __next__：返回容器的一下元素
# __iter__：返回迭代器本身
class CountDown:
    def __init__(self, step):
        self.step = step

    def __next__(self):
        """返回容器的下一个元素"""
        if self.step <= 0:
            raise StopIteration
        self.step -= 1
        return self.step

    def __iter__(self):
        """返回迭代器本身"""
        return self

for i in CountDown(4):
    print(i)

3
2
1
0


### 生成器

In [12]:
# 生成器：使用yield返回结果的一类特殊迭代器(保存执行上下文)
# 一个简单的生成器
def fibonacci():
    a, b = 0, 1

    while True:
        yield b
        a, b = b, b + a

fib = fibonacci()
next(fib)

1

In [13]:
next(fib)

1

In [14]:
next(fib)

2

In [15]:
[next(fib) for i in range(10)]

[3, 5, 8, 13, 21, 34, 55, 89, 144, 233]

In [2]:
# 生成器能够利用next函数与调用的代码交互，其值可以通过send方法来传递
def psychologist():
    print('please tell me your problems')
    i = 0
    while True:
        i += 1
        print(f'第{i}次进入循环')
        answer = yield
        print('answer：', answer)
        if answer is not None:
            if answer.endswith('?'):
                print("don't ask yourself to much questions")
            elif 'good' in answer:
                print('ahh that is good, go on')
            elif 'bad' in answer:
                print('do not be so negative')

free = psychologist() # 调用函数，注意该函数中answer未接受任何传入参数
next(free) # 第一次调用，函数在 `answer = yield` 处暂停

please tell me your problems
第1次进入循环


In [3]:
# send方法将其参数变为yield的返回值，answer接受这个值，程序可以根据传入的值继续逻辑处理
# 进入第二次循环，并在yield处再次暂停
free.send('I feel bad')

answer： I feel bad
do not be so negative
第2次进入循环


In [4]:
# 第二次send，传入值不同，程序依内部处理逻辑进行处理
free.send('你好?')

answer： 你好?
don't ask yourself to much questions
第3次进入循环


In [5]:
# next 与 send 类似，但next直接使yield返回None
next(free)

answer： None
第4次进入循环


### 装饰器

In [6]:
# 装饰器：通常是一个命名的对象（不允许使用lambda表达式），在被（装饰函数）调用时接受单一参数，并返回一个可调用对象
# 任何可调用对象都可以作为装饰器，它们返回的对象可以是实现了__call__方法的对象
# 装饰器语法只是语法糖而已，下面的装饰器用法：
def some_decorator(func):
    def run(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return run


@some_decorator
def decorated_func():
    pass

# 以上可以被替换为显式的装饰器调用和函数的重新赋值：
def decorated_func():
    pass
decorated_func = some_decorator(decorated_func)

In [7]:
# 类装饰器：如果装饰器需要复杂的参数化或者依赖特定状态，则可以使用类装饰器
class DecoratorAsClass:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        # 在调用原始函数前，做点什么
        result = self.func(*args, **kwargs)
        # 在函数调用之后，做点什么
        # 最后返回原始函数的结果
        return result

In [8]:
# 参数化装饰器
def repeat(num=3):
    """多次重复执行装饰函数

    返回最后一个原始函数调用的值作为结果
    :param num: 重复次数，默认值为3
    """
    def actual_decorator(func):
        def wrapper(*args, **kwargs):
            result = None
            for _ in range(num):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return actual_decorator

@repeat(2)
def foo():
    print('foo')

foo()

foo
foo


In [9]:
# 注意，即使参数化装饰器的参数有默认值，名字后面也必须加括号
@repeat()
def bar():
    print('bar')

bar()

bar
bar
bar


In [10]:
# 否则调用装饰器函数会出现错误
@repeat
def bar():
    print('bar')

bar()

TypeError: repeat.<locals>.actual_decorator() missing 1 required positional argument: 'func'

In [11]:
# 保存内省的装饰器
# 使用装饰器的常见错误是在使用装饰器时不保存函数元数据（主要是文档字符串和原始函数名）
def dummy(func):
    def wrapper(*args, **kwargs):
        """包装函数内部文档"""
        return func(*args, **kwargs)
    return wrapper

@dummy
def func_with_important_docstring():
    """这是我们想要保存的文档字符串"""

# 函数func_with_important_docstring失去了它的原始名称和文档字符串
func_with_important_docstring.__name__, func_with_important_docstring.__doc__

('wrapper', '包装函数内部文档')

In [12]:
# 解决方法：使用functools模块内置的wraps()装饰器
from functools import wraps

def preserving_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """包装函数内部文档"""
        return func(*args, **kwargs)
    return wrapper

@preserving_decorator
def func_with_important_docstring():
    """这是我们想要保存的文档字符串"""

#这样定义的装饰器可以保存重要的函数元数据
func_with_important_docstring.__name__, func_with_important_docstring.__doc__

('func_with_important_docstring', '这是我们想要保存的文档字符串')

In [13]:
# 参数检查装器
# 装饰器将函数注册到全局字典，并将其参数和返回值保存在一个类型列表中
rpc_info = {}

def xmlrpc(in_=(), out=(type(None),)):
    def _xmlrpc(func):
        # 注册签名
        func_name = func.__name__
        rpc_info[func_name] = (in_, out)

        def _check_types(elements, types):
            """用来检查类型的子函数"""
            if len(elements) != len(types):
                raise TypeError('argument count is wrong')
            typed = enumerate(zip(elements, types))
            for index, couple in typed:
                arg, of_the_right_type = couple
                if isinstance(arg, of_the_right_type):
                    continue
                raise  TypeError(
                    f'arg #{index} should be {of_the_right_type}'
                )

        # 包装过的函数
        def __xmlrpc(*args): # 没有允许的关键词
            # 检查输入内容
            checkable_args = args[1:]  # 去掉 self
            _check_types(checkable_args, in_)
            # 运行函数
            res = func(*args)
            # 检出输出内容
            if not type(res) in (tuple, list):
                checkable_res = (res,)
            else:
                checkable_res = res
            _check_types(checkable_res, out)

            # 函及其类型检查成功
            return res
        return __xmlrpc
    return _xmlrpc

class RPCView:
    @xmlrpc((int, int))  # two int -> None
    def meth1(self, int1, int2):
        print(f'received {int1} and {int2}')

    @xmlrpc((str,), (int,)) # sting -> int
    def meth2(self, phrase):
        print(f'received {phrase}')
        return 12

In [14]:
# 在实际读取时，这个类定义会填充rpc_info字典，并用于检查参数类型的特定环境中
rpc_info

{'meth1': ((int, int), (NoneType,)), 'meth2': ((str,), (int,))}

In [15]:
my = RPCView()
my.meth1(1, 2)

received 1 and 2


In [16]:
my.meth2(2)

TypeError: arg #0 should be <class 'str'>

In [19]:
# 缓存装饰器
import time
import  hashlib
import pickle


cache = {}

def is_obsolete(entry, duration):
    return time.time() - entry['time'] > duration

def compute_key(func, args, kw):
    key = pickle.dumps((func.__name__, args, kw))
    return hashlib.sha1(key).hexdigest()

def memoize(duration=10):
    def _memorize(func):
        def __memorize(*args, **kw):
            key = compute_key(func, args, kw)

            # 是否拥有它了？
            if (
                key in cache and not is_obsolete(cache[key], duration)
            ):
                print('we got a winner')
                return cache[key]['value']
            # 计算
            res = func(*args, **kw)
            # 保存结果
            cache[key] = {
                'value': res,
                'time': time.time()
            }
            return res
        return __memorize
    return _memorize

@memoize()
def very_very_very_very_very_complex_stuff(a, b):
    return a + b

# 首次调用，没有缓存
very_very_very_very_very_complex_stuff(2, 2)

4

In [20]:
# 10s内相同参数再次调用，使用缓存
very_very_very_very_very_complex_stuff(2, 2)

we got a winner


4

In [24]:
@memoize(1) # 1秒后令缓存失效
def very_very_very_very_very_complex_stuff(a, b):
    return a + b

In [25]:
very_very_very_very_very_complex_stuff(2, 2)

4

In [26]:
very_very_very_very_very_complex_stuff(2, 2)

we got a winner


4

In [27]:
cache

{'ae2095169eb19988a90c4596db1ccd94757ce38c': {'value': 4,
  'time': 1680932733.3544948}}

In [28]:
very_very_very_very_very_complex_stuff(2, 2)

4

In [30]:
# 代理装饰器：使用全局机制来标记和注册函数
# 一个例子：使用集中式检查器和相关的可调用对象要求的权限实现的保护代码访问的安全层
class User(object):
    def __init__(self, roles):
        self.roles = roles

class Unauthorized(Exception):
    pass

def protect(role):
    def _protect(func):
        def __protect(*args, **kw):
            user = globals().get('user')
            if user is None or role not in user.roles:
                raise Unauthorized('I wont tell you')
            return func(*args, *kw)
        return __protect
    return _protect

# 当前用户被保存在一个全局变量中，在方法被访问时装饰器会检查他的角色
tare = User(('admin', 'user'))
bill = User(('user',))

class MySecrets(object):
    @protect('admin')
    def waffle_recipe(self):
        print('use ton of butter!')

these_are = MySecrets()
user = tare
these_are.waffle_recipe()

use ton of butter!


In [31]:
user = bill
these_are.waffle_recipe()

Unauthorized: I wont tell you